In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Deal_with_JSON").master("local[*]").config("spark.ui.port", "4042").getOrCreate()
print(spark.sparkContext.uiWebUrl)
spark

http://gypsum-gpu160.unity.rc.umass.edu:4042


In [5]:
# Read Single line JSON file

df_single = spark.read.format("json").load("data/order_singleline.json")

In [6]:
df_single.printSchema()

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [7]:
df_single.show()

+--------------------+-----------+--------+--------------------+
|             contact|customer_id|order_id|    order_line_items|
+--------------------+-----------+--------+--------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|
+--------------------+-----------+--------+--------------------+



In [8]:
# Read Multiline JSON file

df_multi = spark.read.format("json").option("multiLine", True).load("data/order_multiline.json")

In [9]:
df_multi.printSchema()

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [10]:
df_multi.show()

+--------------------+-----------+--------+--------------------+
|             contact|customer_id|order_id|    order_line_items|
+--------------------+-----------+--------+--------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|
+--------------------+-----------+--------+--------------------+



In [14]:
df = spark.read.format("text").load("data/order_singleline.json")
df.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
df.show()

+--------------------+
|               value|
+--------------------+
|{"order_id":"O101...|
+--------------------+



In [15]:
df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"order_id":"O101","customer_id":"C001","order_line_items":[{"item_id":"I001","qty":6,"amount":102.45},{"item_id":"I003","qty":2,"amount":2.01}],"contact":[9000010000,9000010001]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [17]:
# With Schema

_schema = "customer_id string, order_id string, contact array<long>"

df_schema = spark.read.format("json").schema(_schema).load("data/order_singleline.json")

In [22]:
df_schema.show(truncate=False)

+-----------+--------+------------------------+
|customer_id|order_id|contact                 |
+-----------+--------+------------------------+
|C001       |O101    |[9000010000, 9000010001]|
+-----------+--------+------------------------+



In [25]:
_schema = "contact array<string>, customer_id string, order_id string, order_line_items array<struct<amount double, item_id string, qty long>>"
df_schema_new = spark.read.format("json").schema(_schema).load("data/order_singleline.json")
df_schema_new.printSchema()

root
 |-- contact: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [26]:
df_schema_new.show()

+--------------------+-----------+--------+--------------------+
|             contact|customer_id|order_id|    order_line_items|
+--------------------+-----------+--------+--------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|
+--------------------+-----------+--------+--------------------+



In [28]:
# Function from_json to read from a column

_schema = "contact array<string>, customer_id string, order_id string, order_line_items array<struct<amount double, item_id string, qty long>>"

from pyspark.sql.functions import from_json

df_expanded = df.withColumn("parsed", from_json(df.value, _schema))

In [29]:
df_expanded.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- contact: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_id: string (nullable = true)
 |    |-- order_line_items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- amount: double (nullable = true)
 |    |    |    |-- item_id: string (nullable = true)
 |    |    |    |-- qty: long (nullable = true)



In [31]:
df_expanded.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+
|value                                                                                                                                                                              |parsed                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+
|{"order_id":"O101","customer_id":"C001","order_line_items":[{"item_id":"I001","qty":6,"amount":102.45},{"item_id":"I003","qty":2,"amount":2.01}],"contact":[9000010000,9000010001]}|{[9000010000, 9000010001], C001, O101, [{1

In [32]:
# Function to_json to parse a JSON string
from pyspark.sql.functions import to_json

df_unparsed = df_expanded.withColumn("unparsed", to_json(df_expanded.parsed))

In [33]:
df_unparsed.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- contact: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_id: string (nullable = true)
 |    |-- order_line_items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- amount: double (nullable = true)
 |    |    |    |-- item_id: string (nullable = true)
 |    |    |    |-- qty: long (nullable = true)
 |-- unparsed: string (nullable = true)



In [34]:
df_unparsed.select("unparsed").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|unparsed                                                                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"contact":["9000010000","9000010001"],"customer_id":"C001","order_id":"O101","order_line_items":[{"amount":102.45,"item_id":"I001","qty":6},{"amount":2.01,"item_id":"I003","qty":2}]}|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [35]:
# Get values from Parsed JSON

df_1 = df_expanded.select("parsed.*")

In [36]:
from pyspark.sql.functions import explode

df_2 = df_1.withColumn("expanded_line_items", explode("order_line_items"))

In [37]:
df_2.show()

+--------------------+-----------+--------+--------------------+-------------------+
|             contact|customer_id|order_id|    order_line_items|expanded_line_items|
+--------------------+-----------+--------+--------------------+-------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|  {102.45, I001, 6}|
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|    {2.01, I003, 2}|
+--------------------+-----------+--------+--------------------+-------------------+



In [38]:
df_3 = df_2.select("contact", "customer_id", "order_id", "expanded_line_items.*")

In [45]:
df_3.show(truncate=False)

+------------------------+-----------+--------+------+-------+---+
|contact                 |customer_id|order_id|amount|item_id|qty|
+------------------------+-----------+--------+------+-------+---+
|[9000010000, 9000010001]|C001       |O101    |102.45|I001   |6  |
|[9000010000, 9000010001]|C001       |O101    |2.01  |I003   |2  |
+------------------------+-----------+--------+------+-------+---+



In [40]:
# Explode Array fields
df_final = df_3.withColumn("contact_expanded", explode("contact"))

In [41]:
df_final.printSchema()

root
 |-- contact: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- item_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- contact_expanded: string (nullable = true)



In [46]:
df_final.drop("contact").show()

+-----------+--------+------+-------+---+----------------+
|customer_id|order_id|amount|item_id|qty|contact_expanded|
+-----------+--------+------+-------+---+----------------+
|       C001|    O101|102.45|   I001|  6|      9000010000|
|       C001|    O101|102.45|   I001|  6|      9000010001|
|       C001|    O101|  2.01|   I003|  2|      9000010000|
|       C001|    O101|  2.01|   I003|  2|      9000010001|
+-----------+--------+------+-------+---+----------------+

